# Test Load Daily

Test various aspects of tile-based loading.

## Imports

In [1]:
import os
import numpy as np
from astropy.io import fits
from astropy.table import Table
from sqlalchemy.exc import IntegrityError
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import findfile
import specprodDB.load as db
import specprodDB.tile as t

## Initial Values

In [2]:
specprod = os.environ['SPECPROD'] = 'daily'
overwrite = True
tiles_patch_date = '20241007'

## Initialize Database

In [4]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
db.log = get_logger(DEBUG)
hostname = 'db2-loadbalancer.specprod.production.svc.spin.nersc.org'
# hostname = 'localhost'
postgresql = db.setup_db(schema=specprod, hostname=hostname, username='desi_admin', overwrite=overwrite)
if overwrite:
    db.load_versions('computed', 'daily/v0', 'daily', specprod, 'main')

INFO:load.py:1508:setup_db: Begin creating tables.
INFO:load.py:1511:setup_db: Finished creating tables.
INFO:load.py:1411:load_versions: Loading version metadata.
INFO:load.py:1422:load_versions: Completed loading version metadata.


## Read tiles file

In [16]:
original_tiles_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'tiles-daily-patched-with-kibo-{tiles_patch_date}.csv')
current_tiles_file = findfile('tiles', readonly=True)
tiles_table = Table.read(original_tiles_file, format='ascii.csv')
# tiles_table

DEBUG:meta.py:346:findfile: hpixdir = 'hpix'
DEBUG:meta.py:361:findfile: specprod_dir = '/global/cfs/cdirs/desi/spectro/redux/daily', specprod = 'None'
DEBUG:meta.py:375:findfile: Setting specprod = 'daily'
DEBUG:meta.py:32:get_desi_root_readonly: Using cached _desi_root_readonly=/dvs_ro/cfs/cdirs/desi


In [6]:
current_tiles_file

'/dvs_ro/cfs/cdirs/desi/spectro/redux/daily/tiles-daily.csv'

In [7]:
row_index = np.where((tiles_table['LASTNIGHT'] >= 20201214) & (tiles_table['EFFTIME_SPEC'] > 0))[0]

In [8]:
%%time
candidate_tiles = db.Tile.convert(tiles_table, row_index=row_index)

CPU times: user 696 ms, sys: 11.9 ms, total: 707 ms
Wall time: 706 ms


## Read exposures file

The daily exposures file may contain exposures with `EFFTIME_SPEC == 0`. We do not want to load these. There are also cases where a *tile* has non-zero `EFFTIME_SPEC` but has no *exposures* with non-zero `EFFTIME_SPEC`. At least for now, don't try to load those either.

In [9]:
original_exposures_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], f'exposures-daily-patched-with-kibo-{tiles_patch_date}.fits')
current_exposures_file = findfile('exposures', readonly=True)
exposures_table = Table.read(original_exposures_file, format='fits', hdu='EXPOSURES')
frames_table = Table.read(original_exposures_file, format='fits', hdu='FRAMES')
# exposures_table[exposures_table['TILEID'] == new_tile.tileid]

DEBUG:meta.py:346:findfile: hpixdir = 'hpix'
DEBUG:meta.py:361:findfile: specprod_dir = '/global/cfs/cdirs/desi/spectro/redux/daily', specprod = 'None'
DEBUG:meta.py:375:findfile: Setting specprod = 'daily'
DEBUG:meta.py:32:get_desi_root_readonly: Using cached _desi_root_readonly=/dvs_ro/cfs/cdirs/desi


In [10]:
%%time
load_tiles = list()
bad_tiles = list()
load_exposures = list()
for new_tile in candidate_tiles:
    row_index = np.where((exposures_table['TILEID'] == new_tile.tileid) & (exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(exposures_table, row_index=row_index)
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((exposures_table['TILEID'] == new_tile.tileid))[0]
        print(exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        bad_tiles.append(new_tile)
# load_exposures

CPU times: user 30.6 s, sys: 206 ms, total: 30.8 s
Wall time: 30.9 s


In [11]:
%%time
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(frames_table, row_index=row_index)
# load_frames

CPU times: user 3min 17s, sys: 1.13 s, total: 3min 18s
Wall time: 3min 18s


In [12]:
%%time
try:
    db.dbSession.add_all(load_tiles)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 2.7 s, sys: 128 ms, total: 2.83 s
Wall time: 3.52 s


In [13]:
%%time
try:
    db.dbSession.add_all(load_exposures)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 3.66 s, sys: 104 ms, total: 3.77 s
Wall time: 7.39 s


In [14]:
%%time
try:
    db.dbSession.add_all(load_frames)
    db.dbSession.commit()
except IntegrityError as exc:
    print(exc.args[0])
    db.dbSession.rollback()

CPU times: user 47.3 s, sys: 972 ms, total: 48.2 s
Wall time: 1min 13s


## Test tile-based Updates

Useful links:

* https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-queryguide-upsert
* https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#sqlalchemy.dialects.postgresql.Insert.on_conflict_do_update.params.set_
* https://docs.sqlalchemy.org/en/20/tutorial/orm_data_manipulation.html#tutorial-orm-data-manipulation

In [23]:
import json
with open(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-cache.json'), 'w') as j:
    json.dump(dict(zip(tiles_table['TILEID'].tolist(), tiles_table['UPDATED'].tolist())), j)

In [24]:
with open(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-cache.json')) as j:
    tiles_cache = json.load(j)

In [19]:
update_tiles_table = Table.read(current_tiles_file, format='fits', hdu='TILES')

In [32]:
cached_tiles = np.array(list(map(int, tiles_cache.keys())))
new_tiles = ~np.in1d(update_tiles_table['TILEID'], cached_tiles)
updated_tiles = np.zeros((len(update_tiles_table), ), dtype=bool)
for tileid in tiles_cache:
    t = int(tileid)
    w = np.where(update_tiles_table['TILEID'] == t)[0]
    assert len(w) == 1
    if update_tiles_table['UPDATED'][w] == tiles_cache[tileid]:
        pass
    elif update_tiles_table['UPDATED'][w] > tiles_cache[tileid]:
        # print("{0} > {1}".format(update_tiles_table['UPDATED'][w], tiles_cache[tileid]))
        updated_tiles[w] = True
    else:
        print("Something weird happened.")
update_tiles_table[new_tiles]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT,UPDATED
int64,bytes20,bytes6,bytes20,bytes20,int64,float64,float64,float64,float64,float64,float64,float64,bytes20,float64,float64,float64,float64,bytes20,float64,int64,bytes24
6498,main,dark,dark,maindark,1,949.7,20.467,28.44,1013.4,1019.5,1050.2,1000.0,obsend,1019.5,1078.6,983.0,1354.0,dark,0.85,20241007,2024-10-08T21:31:03-0700
8122,main,dark,dark,maindark,1,1646.5,331.757,-13.178,1006.0,1122.4,1153.2,1000.0,obsend,1122.4,1122.3,1134.1,1648.2,dark,0.85,20241007,2024-10-08T21:31:03-0700
6757,main,dark,dark,maindark,1,865.6,2.874,27.131,1008.7,968.6,1079.3,1000.0,obsend,968.6,989.3,951.2,1410.0,dark,0.85,20241007,2024-10-08T21:31:03-0700
6501,main,dark,dark,maindark,1,1084.4,28.043,28.166,1009.9,1032.8,1052.0,1000.0,obsend,1032.8,1076.8,1006.2,1395.9,dark,0.85,20241007,2024-10-08T21:31:03-0700
6225,main,dark,dark,maindark,1,1487.1,336.749,-12.178,1006.3,1105.7,1098.3,1000.0,obsend,1105.7,1114.0,1108.5,1551.7,dark,0.85,20241007,2024-10-08T21:31:03-0700
1518,main,dark,dark,maindark,1,1439.5,347.602,-12.842,1009.4,1051.2,1077.8,1000.0,obsend,1051.2,1055.7,1063.2,1506.5,dark,0.85,20241007,2024-10-08T21:31:03-0700
2471,main,dark,dark,maindark,1,1330.3,37.973,-17.363,1002.7,1075.4,1021.0,1000.0,obsend,1075.4,1087.1,1081.5,1533.5,dark,0.85,20241007,2024-10-08T21:31:03-0700
1867,main,dark,dark,maindark,1,1648.9,327.328,-10.39,1004.7,1117.2,1112.6,1000.0,obsend,1117.2,1128.8,1124.2,1598.3,dark,0.85,20241007,2024-10-08T21:31:03-0700
1804,main,dark,dark,maindark,1,1078.6,30.674,25.605,1004.0,1065.7,1048.9,1000.0,obsend,1065.7,1129.3,1020.2,1284.4,dark,0.85,20241007,2024-10-08T21:31:03-0700


In [33]:
update_tiles_table[updated_tiles]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT,UPDATED
int64,bytes20,bytes6,bytes20,bytes20,int64,float64,float64,float64,float64,float64,float64,float64,bytes20,float64,float64,float64,float64,bytes20,float64,int64,bytes24
22957,main,bright,bright,mainbright,3,1065.7,128.532,30.959,192.7,180.0,214.5,180.0,obsend,157.4,172.7,180.0,206.2,bright,0.85,20241007,2024-10-08T21:31:03-0700
27443,main,bright,bright,mainbright,2,964.4,115.927,28.387,157.4,184.0,192.1,180.0,obsend,167.5,196.2,184.0,188.4,bright,0.85,20241007,2024-10-08T21:31:03-0700


In [36]:
load_new_tiles = db.Tile.convert(update_tiles_table, row_index=new_tiles)
load_updated_tiles = db.Tile.convert(update_tiles_table, row_index=updated_tiles)

### Find exposures associated with new and updated tiles

In [35]:
update_exposures_table = Table.read(current_exposures_file, format='fits', hdu='EXPOSURES')
update_frames_table = Table.read(current_exposures_file, format='fits', hdu='FRAMES')

In [37]:
load_tiles = list()
load_exposures = list()
for new_tile in (load_new_tiles + load_updated_tiles):
    row_index = np.where((update_exposures_table['TILEID'] == new_tile.tileid) & (update_exposures_table['EFFTIME_SPEC'] > 0))[0]
    if len(row_index) > 0:
        load_tiles.append(new_tile)
        load_exposures += db.Exposure.convert(update_exposures_table, row_index=row_index)
    else:
        print("ERROR: No valid exposures found for tile {0:d}, even though EFFTIME_SPEC == {1:f}!".format(new_tile.tileid, new_tile.efftime_spec))
        bad_index = np.where((update_exposures_table['TILEID'] == new_tile.tileid))[0]
        print(update_exposures_table[['EXPID', 'NIGHT', 'MJD', 'EFFTIME_SPEC']][bad_index])
        # bad_tiles.append(new_tile)
# load_tiles, load_exposures

([Tile(tileid=6498),
  Tile(tileid=8122),
  Tile(tileid=6757),
  Tile(tileid=6501),
  Tile(tileid=6225),
  Tile(tileid=1518),
  Tile(tileid=2471),
  Tile(tileid=1867),
  Tile(tileid=1804),
  Tile(tileid=1549),
  Tile(tileid=8259),
  Tile(tileid=5381),
  Tile(tileid=8261),
  Tile(tileid=9123),
  Tile(tileid=8495),
  Tile(tileid=8496),
  Tile(tileid=8725),
  Tile(tileid=8732),
  Tile(tileid=9745),
  Tile(tileid=9763),
  Tile(tileid=11376),
  Tile(tileid=20973),
  Tile(tileid=27138),
  Tile(tileid=1516),
  Tile(tileid=27152),
  Tile(tileid=8264),
  Tile(tileid=22957),
  Tile(tileid=27443)],
 [Exposure(night=20241007, expid=256820, tileid=6498),
  Exposure(night=20241007, expid=256808, tileid=8122),
  Exposure(night=20241007, expid=256816, tileid=6757),
  Exposure(night=20241007, expid=256823, tileid=6501),
  Exposure(night=20241007, expid=256809, tileid=6225),
  Exposure(night=20241007, expid=256811, tileid=1518),
  Exposure(night=20241007, expid=256824, tileid=2471),
  Exposure(night=202

In [96]:
load_frames = list()
for exposure in load_exposures:
    row_index = np.where(update_frames_table['EXPID'] == exposure.expid)[0]
    assert len(row_index) > 0
    load_frames += db.Frame.convert(update_frames_table, row_index=row_index)

In [76]:
stmt = db.upsert(load_tiles)
# print(stmt)
db.dbSession.execute(stmt)
db.dbSession.commit()

In [90]:
stmt = db.upsert(load_exposures)
# print(stmt)
db.dbSession.execute(stmt)
db.dbSession.commit()

INSERT INTO daily.exposure (night, expid, tileid, tilera, tiledec, date_obs, mjd, survey, program, faprgrm, faflavor, exptime, efftime_spec, goaltime, goaltype, mintfrac, airmass, ebv, seeing_etc, efftime_etc, tsnr2_elg, tsnr2_qso, tsnr2_lrg, tsnr2_lya, tsnr2_bgs, tsnr2_gpbdark, tsnr2_gpbbright, tsnr2_gpbbackup, lrg_efftime_dark, elg_efftime_dark, bgs_efftime_bright, lya_efftime_dark, gpb_efftime_dark, gpb_efftime_bright, gpb_efftime_backup, transparency_gfa, seeing_gfa, fiber_fracflux_gfa, fiber_fracflux_elg_gfa, fiber_fracflux_bgs_gfa, fiberfac_gfa, fiberfac_elg_gfa, fiberfac_bgs_gfa, airmass_gfa, sky_mag_ab_gfa, sky_mag_g_spec, sky_mag_r_spec, sky_mag_z_spec, efftime_gfa, efftime_dark_gfa, efftime_bright_gfa, efftime_backup_gfa) VALUES (%(night_m0)s, %(expid_m0)s, %(tileid_m0)s, %(tilera_m0)s, %(tiledec_m0)s, %(date_obs_m0)s, %(mjd_m0)s, %(survey_m0)s, %(program_m0)s, %(faprgrm_m0)s, %(faflavor_m0)s, %(exptime_m0)s, %(efftime_spec_m0)s, %(goaltime_m0)s, %(goaltype_m0)s, %(mintfrac_m

In [97]:
stmt = db.upsert(load_frames)
# print(stmt)
db.dbSession.execute(stmt)
db.dbSession.commit()

## Load photometry for the tile

When tractor photometry is written out by John Moustakas' VAC code, only objects with `brickname != ''` are written.

In [ ]:
tile_index = 0

In [ ]:
load_tiles = db.dbSession.query(db.Tile).filter(db.Tile.tileid == tile_id).all()

In [ ]:
load_tiles[tile_index] #, load_tiles[tile_index + 1], load_tiles[-2], load_tiles[-1]

In [ ]:
%%time
potential_targets_table = t.potential_targets(load_tiles[tile_index].tileid)
# potential_targets_table

In [ ]:
%%time
potential_cat = t.potential_photometry(load_tiles[tile_index], potential_targets_table)
# potential_cat

In [ ]:
%%time
potential_targetphot = t.targetphot(potential_cat)

In [ ]:
%%time
potential_tractorphot = t.tractorphot(potential_cat)

In [ ]:
%%time
load_photometry = t.load_photometry(potential_tractorphot)

### Load photometry, such as it is, for objects that are not in the tractor catalog

In [ ]:
%%time
load_targetphot = t.load_targetphot(potential_targetphot, load_photometry)

### Load the target table

In [ ]:
%%time
load_target = t.load_target(load_tiles[tile_index], potential_targetphot)

## Load tile/cumulative redshifts

Need a way to compute "best" spectra as new tiles are added. There are a lot of columns that come from other sources here, so need to track these down.

In [ ]:
%%time
load_ztile = t.load_redshift(load_tiles[tile_index])

In [ ]:
l = db.dbSession.query(db.Ztile).filter(db.Ztile.tileid == tile_id).all()

In [ ]:
len(l)

In [ ]:
l[0]

In [ ]:
l[0].z, l[0].zerr, l[0].zwarn, l[0].spectype, l[0].subtype

## Load fiberassign and potential

In [ ]:
%%time
load_fiberassign, load_potential = t.load_fiberassign(load_tiles[tile_index])

## Recompute global values

The global values are the primary classification and number of spectra.

In [ ]:
t.update_primary()

## q3c Update

`tile`, `exposure`, `photometry`, `fiberassign`

In [ ]:
t.update_q3c()

In [98]:
db.dbSession.close()
db.engine.dispose()